# Load Data

In [2]:
import json
import mysql.connector

In [3]:
photos = []
for line in open('../../../yelp_academic_dataset_photo.json', 'r'):
    photos.append(json.loads(line))

In [4]:
photos[0]

{'photo_id': 'IuXwafFH3fZlTyXA-poz0w',
 'business_id': 'wRKYaVXTks43GVSI2awTQA',
 'caption': '',
 'label': 'food'}

In [7]:
len(photos)

280992

In [8]:
business = []
for line in open('../../../yelp_academic_dataset_business.json', 'r'):
    business.append(json.loads(line))

In [9]:
business[0]

{'business_id': 'Apn5Q_b6Nz61Tq4XzPdf9A',
 'name': 'Minhas Micro Brewery',
 'neighborhood': '',
 'address': '1314 44 Avenue NE',
 'city': 'Calgary',
 'state': 'AB',
 'postal_code': 'T2E 6L6',
 'latitude': 51.0918130155,
 'longitude': -114.031674872,
 'stars': 4.0,
 'review_count': 24,
 'is_open': 1,
 'attributes': {'BikeParking': 'False',
  'BusinessAcceptsCreditCards': 'True',
  'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
  'GoodForKids': 'True',
  'HasTV': 'True',
  'NoiseLevel': 'average',
  'OutdoorSeating': 'False',
  'RestaurantsAttire': 'casual',
  'RestaurantsDelivery': 'False',
  'RestaurantsGoodForGroups': 'True',
  'RestaurantsPriceRange2': '2',
  'RestaurantsReservations': 'True',
  'RestaurantsTakeOut': 'True'},
 'categories': 'Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel',
 'hours': {'Monday': '8:30-17:0',
  'Tuesday': '11:0-21:0',
  'Wednesday': '11:0-21:0',
  'Thursday': '11:0-21:0',
  'Friday'

In [10]:
len(business)

188593

# connection to database

In [1]:
mydb = mysql.connector.connect(
  host="****************",
  user="***********",
  passwd="*************",
  database="*******"
)
mycursor = mydb.cursor()

NameError: name 'mysql' is not defined

In [ ]:
# design database schema here

In [21]:
sql_init = """
create table Business(
  business_id varchar(22) primary key,
  b_name text,
  neighborhood text,
  city text,
  state text,
  postal_code varchar(7),
  latitude float,
  longitude float,
  stars numeric,
  review_count int,
  is_open int
);

create table Photos(
  photo_id varchar(22),
  business_id varchar (22) references Business,
  label text,
  primary key (photo_id, business_id) 
);
"""


In [22]:
iterator = mycursor.execute(sql_init, multi=True)

In [25]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

('Business',)
('Photos',)


In [54]:
len(business)

188593

In [87]:
value = list(map(lambda record: (record['business_id'], 
                         record['name'],
                         record['neighborhood'],
                         record['city'],
                         record['state'],
                         record['postal_code'],
                         record['latitude'],
                         record['longitude'],
                         record['stars'],
                         record['review_count'],
                         record['is_open']) ,business[170000:188593]))

In [88]:
sql_insert = """INSERT INTO Business (business_id,
                               b_name, 
                               neighborhood, 
                               city, 
                               state,
                               postal_code,
                               latitude,
                               longitude,
                               stars,
                               review_count,
                               is_open)
                               VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""


mycursor.executemany(sql_insert, value)

In [89]:
mydb.commit()
print(mycursor.rowcount, "record inserted.")

18593 record inserted.


In [98]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [115]:
for x in batch(range(0, len(photos)), 30000):
    sql_insert = """INSERT INTO Photos (photo_id,
                               business_id, 
                               label)
                               VALUES (%s,%s,%s)"""

    value = list(map(lambda record: (record['photo_id'], 
                         record['business_id'],
                         record['label']) ,photos[slice(x.start,x.stop,1)]))

    mycursor.executemany(sql_insert, value)
    mydb.commit()
    print(mycursor.rowcount, "record inserted.")

30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
30000 record inserted.
10992 record inserted.
